In [1]:
import pandas as pd
import numpy as np
import pickle
import logging
import plotly.express as px
from sklearn.model_selection import train_test_split
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
from cluster import cluster_movies
import logging
logging_format = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(format=logging_format, datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
seed = 50000

In [22]:
from EDA_functions import clean_string

# Exploración del contenido de la data.

De acuerdo a la documentación disponible de esta base de datos (vía: https://files.grouplens.org/datasets/movielens/ml-20m-README.html) se llega a que:
* *links.cv* no se usará debido a que solo son identificadores de las películas en las páginas.
* *genome-scores.csv* y *genome-tags.csv*. La columna que ofrecería cierta información razonable sería `relavance`, pero, la descripción dice lo siguiente: *"the tag genome was computed using a machine learning algorithm on user-contributed content including tags, ratings, and textual reviews"*. Se entiende que ha utilizado información de toda la data del conjunto, por lo que considerar estos scores como features para el entrenamieto y prueba estaría provocando fuga de información (leakege) de uno a otro conjunto. Entonces, no se usarán estos scores.  

Se cargan las datas de interés y se hacen una serie de depliegues de las primeras filas de cada una para conocer el contenido.

In [81]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')
tag = pd.read_csv('tag.csv')
logging.info("BASE DE DATOS: movie")
display(movie.head(3))
display(movie.shape)
print("Presencia de missing:")
display(movie.isna().sum())
logging.info("BASE DE DATOS: rating")
display(rating.head(3))
display(rating.shape)
print("Presencia de missing:")
display(rating.isna().sum())
logging.info("BASE DE DATOS: tag")
display(tag.head(3))
display(tag.shape)
print("Presencia de missing:")
display(tag.isna().sum())

2021-11-28 19:55:45 INFO: BASE DE DATOS: movie


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


(27278, 3)

Presencia de missing:


movieId    0
title      0
genres     0
dtype: int64

...

2021-11-28 19:55:46 INFO: BASE DE DATOS: tag


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19


(465564, 4)

Presencia de missing:


userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

Con base en estadísticios resumen construidos de manera independiente. Se tienen los siguientes insights con respecto a la data.

Para los 138,493 __usuarios__ que aparecen en *rating.csv* se tiene que:
* Todos ellos han revisado como mínimo 20 películas.
* Uno de ellos ha revisado 9,254 películas.  
* Sólo 7,801 han dejado dejado algún tipo de tags en las películas.

Para las 27,278 __películas__ que aparecen en *movies.csv*.
* 19,545 tiene algún tipo de *token*: palabra relevante en los tags obtenidas con ayuda de procesamiento de lenguaje natural.
* Solo aparecen 26,744 en la data *rating.csv*, de las cuales:

    1) 3,972 solo tiene 1 reseña. \
    2) 2,043 solo tiene 2 reseñas. \
    3) 1,355 solo tiene 3 reseñas. \
    ... \
    10) 372 solo tiene 10 reseñas. \
    ... 

La estructura de esta data es natural para Sistemas de Recomendación basados en Filtrado Colaborativo. Además, hay una indexación en el tiempo. Habrá que adaptar la estructura de esta base de datos a una típica de tratar con modelos de aprendizaje supervisado.

Para tal fin, dividiremos los __usuarios__ de una vez en conjuntos de entrenamiento y prueba. Esto hará que la información en el tiempo de cierto usuario en el entrenamiento no aparezca en el test y viceversa.

Con base en lo anterior, __el modelo que se preparará predecirá para nuevos usuarios que comiencen a hacer historial de *rating* dada una lista conocida de películas__.

In [71]:
logging.info('SE DIVIDEN LOS USUARIOS EN CONJUNTOS DE PRE-ENTRENAMIENTO Y PRE-PRUEBA.')
total_user = rating[['userId']].drop_duplicates()
total_user.reset_index(drop=True,
                       inplace=True)
user_train, user_test = train_test_split(total_user,
                                         test_size=0.30,
                                         random_state=seed,
                                         shuffle=True)

2021-11-28 18:05:04 INFO: SE DIVIDEN LOS USUARIOS EN CONJUNTOS DE PRE-ENTRENAMIENTO Y PRE-PRUEBA.


# Conjuntos de Entrenamiento y Prueba.

Con el fin de no saturar los recursos computaciones que tenemos a disposición. Tomaremos el 50% de cada uno de esos conjuntos.

In [4]:
logging.info('SE SUBMUESTREAN LOS CONJUNTOS DE USUARIOS EN ENTRENAMIENTO Y DE PRUEBA.')
user_train_1, user_train_2 = train_test_split(user_train,
                                              test_size=0.50,
                                              random_state=seed,
                                              shuffle=True)
user_test_1, user_test_2 = train_test_split(user_test,
                                            test_size=0.50,
                                            random_state=seed,
                                            shuffle=True)
list_user_test_1=list(user_test_1['userId'])
list_user_train_2=list(user_train_2['userId'])
rating_redundant_test_1 = rating[rating['userId'].isin(list_user_test_1)]
rating_redundant_train_2 = rating[rating['userId'].isin(list_user_train_2)]
rating_redundant_test_1.sort_values(by='timestamp',
                                    ascending=False,
                                    inplace=True)
rating_redundant_test_1.reset_index(drop=True,
                                    inplace=True)
rating_redundant_train_2.sort_values(by='timestamp',
                                     ascending=False,
                                     inplace=True)
rating_redundant_train_2.reset_index(drop=True,
                                     inplace=True)

2021-11-28 14:31:37 INFO: SE SUBMUESTREAN LOS CONJUNTOS DE USUARIOS DE ENTRENAMIENTO Y DE PRUEBA.


Se tienen entonces dos bases de datos, el entrenamiento, llamado `rating_redundant_train_2`, y el de prueba, es decir, `rating_redundant_test_1`. La palabra `redundant` tiene un porqué y se explicará más adelante. 

Para estas dos datas, tenemos los siguientes insights:

In [70]:
users_train_2 = rating_redundant_train_2['userId'].unique()
count_users_train_2 = len(users_train_2 )
users_test_1 = rating_redundant_test_1['userId'].unique()
count_users_test_1= len(users_test_1)
movies_train_2 = rating_redundant_train_2['movieId'].unique()
count_movies_train_2 = len(movies_train_2 )
movies_test_1 = rating_redundant_test_1['movieId'].unique()
count_movies_test_1= len(movies_test_1)
logging.info(f'TOTAL DE DATA EN ENTRENAMIENTO: {rating_redundant_train_2.shape[0]}')
logging.info(f'TOTAL DE PELÍCULAS EN ENTRENAMIENTO: {count_movies_train_2}')
logging.info(f'TOTAL DE USUARIOS EN ENTRENAMIENTO: {count_users_train_2}')
logging.info(f'TOTAL DE DATA EN PRUEBA: {rating_redundant_test_1.shape[0]}')
logging.info(f'TOTAL DE PELÍCULAS EN PRUEBA: {count_movies_test_1}')
logging.info(f'TOTAL DE USUARIOS EN PRUEBA: {count_users_test_1}')

2021-11-28 18:03:01 INFO: TOTAL DE DATA EN ENTRENAMIENTO: 6985826
2021-11-28 18:03:01 INFO: TOTAL DE PELÍCULAS EN ENTRENAMIENTO: 22540
2021-11-28 18:03:01 INFO: TOTAL DE USUARIOS EN ENTRENAMIENTO: 48473
2021-11-28 18:03:01 INFO: TOTAL DE DATA EN PRUEBA: 3014377
2021-11-28 18:03:01 INFO: TOTAL DE PELÍCULAS EN PRUEBA: 20132
2021-11-28 18:03:01 INFO: TOTAL DE USUARIOS EN PRUEBA: 20774


NOTA: Será interesante saber cómo se comportará el futuro modelo de clasificación para los __usuarios__ en el conjunto de prueba cuyas __películas__ no están en el conjunto de entrenamiento. Se guardarán los `userId`'s y `movieId`'s en esta situación más adelate.

In [10]:
movies_not_in_train = list(set(movies_test_1) - set(movies_train_2))
len_movies_not_in_train = len(movies_not_in_train)
logging.info(f'EN TOTAL HAY {len_movies_not_in_train} PELÍCULAS QUE NO ESTÁN PRESENTES EN EL ENTRENAMIENTO.')

2021-11-28 14:48:19 INFO: EN TOTAL HAY 1817 PELÍCULAS QUE NO ESTÁN PRESENTES EN EL ENTRENAMIENTO.


# Creaciación de variables objetivo, fecha y año.

A manera de insight, esta es la distribución global de `rating`.

In [11]:
rating_distribution = (rating['rating'].value_counts(normalize=True)
                                       .to_frame('percentage')
                                       .reset_index())
rating_distribution.columns = ['rating', 'percentage']
rating_distribution['percentage'] = 100 * rating_distribution['percentage']
rating_distribution.sort_values(by='rating', ascending=False, inplace=True)
logging.info(f'DISTRIBUCIÓN DEL RATING.')
display(rating_distribution.reset_index(drop=True))

2021-11-28 14:49:11 INFO: DISTRIBUCIÓN DEL RATING.


,rating,percentage
0,5.0,14.493109
1,4.5,7.674019
2,4.0,27.809264
3,3.5,11.000635
4,3.0,21.455683
5,2.5,4.416932
6,2.0,7.154891
7,1.5,1.396242
8,1.0,3.403615
9,0.5,1.195609


Se crea variable objetivo, en el cual `1` será el caso cuando el *rating* sea mayor o igual a 4 (*high rating*), y 0 caso contario. Notar que, de manera global, un `high rating` representa cerca de un % 50.4.
La creación de la variable objetivo sucederá en los conjuntos de entrenamiento y prueba.

In [12]:
logging.info('SE CREA VARIABLE OBJETIVO EN ENTRENAMIENTO Y PRUEBA.')
rating_redundant_test_1['high_rating'] = rating_redundant_test_1.apply(lambda row: 1 if row['rating'] >= 4 else 0,
                                                                       axis=1)
rating_redundant_train_2['high_rating'] = rating_redundant_train_2.apply(lambda row: 1 if row['rating'] >= 4 else 0,
                                                                         axis=1)
logging.info('¡LISTO!')
logging.info(' DISTRIBUCIÓN DE HIGH RATING EN ENTRENAMIENTO.')
train_rating_dist = (rating_redundant_train_2['high_rating'].value_counts(normalize=True)
                                                            .to_frame('percentage')
                                                            .reset_index())
train_rating_dist.columns = ['high_rating', 'percentage']
train_rating_dist['percentage'] = 100 * train_rating_dist['percentage']
train_rating_dist.sort_values(by='high_rating', ascending=False, inplace=True)
display(train_rating_dist.reset_index(drop=True))
logging.info(' DISTRIBUCIÓN DE HIGH RATING EN PRUEBA.')
test_rating_dist = (rating_redundant_test_1['high_rating'].value_counts(normalize=True)
                                                          .to_frame('percentage')
                                                          .reset_index())
test_rating_dist.columns = ['high_rating', 'percentage']
test_rating_dist['percentage'] = 100 * test_rating_dist['percentage']
test_rating_dist.sort_values(by='high_rating', ascending=False, inplace=True)
display(test_rating_dist.reset_index(drop=True))

2021-11-28 14:49:18 INFO: SE CREA VARIABLE OBJETIVO EN ENTRENAMIENTO y PRUEBA.
2021-11-28 14:50:29 INFO: ¡LISTO!
2021-11-28 14:50:29 INFO:  DISTRIBUCIÓN DE HIGH RATING EN ENTRENAMIENTO.


,high_rating,percentage
0,1,50.048155
1,0,49.951845


2021-11-28 14:50:29 INFO:  DISTRIBUCIÓN DE HIGH RATING EN PRUEBA.


,high_rating,percentage
0,1,49.725996
1,0,50.274004


Se obtendrá la fecha, a partir de ahora llamada `time_day`, y el año, es decir, `year` de cada una de las revisiones hechas por los usuarios. Esto se hará de manera preliminar, es decir, sólo para que ya estén presentes en la data.

In [14]:
logging.info('SE OBTIENE FECHA y AÑO DE RANKEO.')
rating_redundant_test_1['time_day'] = rating_redundant_test_1.apply(lambda row: row['timestamp'].split()[0], axis=1)
rating_redundant_test_1['time_day'] = pd.to_datetime(rating_redundant_test_1['time_day'])
rating_redundant_test_1['timestamp'] = pd.to_datetime(rating_redundant_test_1['timestamp'])
rating_redundant_test_1['year'] = pd.DatetimeIndex(rating_redundant_test_1['time_day']).year
rating_redundant_train_2['time_day'] = rating_redundant_train_2.apply(lambda row: row['timestamp'].split()[0], axis=1)
rating_redundant_train_2['time_day'] = pd.to_datetime(rating_redundant_train_2['time_day'])
rating_redundant_train_2['timestamp'] = pd.to_datetime(rating_redundant_train_2['timestamp'])
rating_redundant_train_2['year'] = pd.DatetimeIndex(rating_redundant_train_2['time_day']).year
logging.info('¡LISTO!')

2021-11-28 14:54:30 INFO: SE OBTIENE FECHA y AÑO DE RANKEO.
2021-11-28 14:55:47 INFO: ¡LISTO!


In [17]:
logging.info('SE GUARDAN DATAS DE ENTRENAMIENTO Y PRUEBA SIN FEATURES.')
# path_test_1 = 'rating_redundant_test_1.sav'
# pickle.dump(rating_redundant_test_1, open(path_test_1, 'wb'))
# path_train_2 = 'rating_redundant_train_2.sav'
# pickle.dump(rating_redundant_train_2, open(path_train_2, 'wb'))

2021-11-28 15:14:22 INFO: SE GUARDAN DATAS DE ENTRENAMIENTO Y PRUEBA SIN FEATURES.


# Features para codificar Películas.

Observamos algunos ejemplos de __películas__ que se tienen en el total. 

In [25]:
movie.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


La variable `genres` puede ayudarnos a dar las primeras features que tengan que ver con las características de la __películas__. En total se han identificado los siguientes géneros en la totalidad de la data.
* film-noir
* no genres listed (también se toma como grupo)
* drama
* mystery 
* animation
* horror
* genre_fantasy
* war
* crime
* comedy
* western
* genre_adventure
* documentary
* imax
* action
* children
* musical
* thriller
* romance
* sci-fi

Se generarán variables dummy (binarias) para cada uno de estos 20 géneros. Una película tendrá un `1` en estas columnas dependiendo al conjunto de genéros a los que pertenece y cero en caso contrario. Para el ejemplo pequeño anterior, tendríamos:

In [29]:
small_movie = movie.head(5)
small_movie['genres_list'] = list(map(clean_string,
                                  small_movie['genres']))
list_genres = list(small_movie['genres_list'])
total_genres = set().union(*list_genres)
total_genres = list(total_genres)
for genre in total_genres:
    small_movie[f'genre_{genre}'] = small_movie.apply(lambda row: 1 if genre in row['genres_list'] else 0,
                                                      axis=1)
logging.info('SE GENERAN VARIABLES DUMMY PARA EL PEQUEÑO CONJUNTO DE PELÍCULAS DE EJEMPLO.')
small_movie.drop(['movieId', 'genres_list'], axis=1)

2021-11-28 15:49:12 INFO: SE GENERAN VARIABLES DUMMY PARA EL PEQUEÑO CONJUNTO DE PELÍCULAS DE EJEMPLO.


,title,genres,genre_children,genre_romance,genre_animation,genre_adventure,genre_comedy,genre_fantasy,genre_drama
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,0,1,1,1,1,0
1,Jumanji (1995),Adventure|Children|Fantasy,1,0,0,1,0,1,0
2,Grumpier Old Men (1995),Comedy|Romance,0,1,0,0,1,0,0
3,Waiting to Exhale (1995),Comedy|Drama|Romance,0,1,0,0,1,0,1
4,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0


Ahora bien, se puede medir el __grado de similaridad__ de estas películas con ayuda justamente de las columnas dummy de géneros. Esto nos ayudaría a clusterizar las películas muy parecidas y ayudarnos a *eliminar la redundancia*. Para eso, podemos usar un cluster herárquico. Este se hace usando la función `cluster_movies` contenida en el script `cluster.py`. El número de clusters que se identifican en arbitrario. En esta ocasión, se ha escogido localizar 15 grandes clusters en las películas; no ha habido una razón en particular, solo experimentación.

In [34]:
# movie_with_cluster = cluster_movies(data_movie=movie,
#                                     n_clusters=15)
movie_with_cluster = pickle.load(open('movie_with_cluster.sav',
                                      'rb'))

Por ejemplo, si se quiere ver a los integrantes del Cluster Número 1, verá, por ejemplo, a los siguientes:

In [37]:
movie_with_cluster[movie_with_cluster['cluster']==1][['title', 'genres', 'cluster']].head(10)

,title,genres,cluster
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,Jumanji (1995),Adventure|Children|Fantasy,1
7,Tom and Huck (1995),Adventure|Children,1
12,Balto (1995),Adventure|Animation|Children,1
26,Now and Then (1995),Children|Drama,1
33,Babe (1995),Children|Drama,1
37,It Takes Two (1995),Children|Comedy,1
47,Pocahontas (1995),Animation|Children|Drama|Musical|Romance,1
52,Lamerica (1994),Adventure|Drama,1
53,"Big Green, The (1995)",Children|Comedy,1


El anterior cluster se visuliza bastante razonable. Así, se tendrá a `cluster` como una nueva feature.

NOTA: Se ha tomado como *__supuesto__* que se conoce en todo momento el listado cumpleto de las __películas__ a las que los usuarios van a otorgar un `rating`. Siendo así, es posible tomar TODAS las películas de *movie.csv* (y por ende, algunas de ellas están en el conjunto de entrenamiento y prueba) y clusterizar. 

NOTA: Es posible trabajar con películas nuevas, es decir, que solo estén en el conjunto de prueba. Para eso, se toman todas las películas que estén en el conjunto de entrenamiento, se clusterizan, y luego, con ayuda de una medida de *similaridad* (la del Coseno, Jaccard o la de Levenshtein), se obteine, para una película no presente en el entrenamiento, al más perecido perteneciente a este último conjunto, y se asigna el cluster correspondiente. Por falta de tiempo, no se pudo desarrollar esta idea.

### ELIMINACIÓN DE INFORMACIÓN REDUNDANTE DE PELÍCULAS EN LA DATA DE ENTRENAMIENTO.

La data de entrenamiento tiene millones de usuarios que han puesto rating a 2,540 películas.

In [40]:
logging.info('DESPLIEGUE DE DATA DE ENTRENAMIENTO.')
rating_redundant_train_2_with_cluster = pd.merge(rating_redundant_train_2,
                                                 movie_with_cluster,
                                                 how="left",
                                                 on=["movieId"])
rating_redundant_train_2_with_cluster

2021-11-28 16:52:30 INFO: DESPLIEGUE DE DATA DE ENTRENAMIENTO.


,userId,movieId,rating,timestamp,high_rating,time_day,year,title,genres,genres_list,...,genre_musical,genre_sci-fi,genre_animation,genre_fantasy,genre_children,genre_western,genre_war,genre_mystery,genre_film-noir,cluster
0,87586,7151,3.5,2015-03-31 06:40:02,0,2015-03-31,2015,Girl with a Pearl Earring (2003),Drama|Romance,"[drama, romance]",...,0,0,0,0,0,0,0,0,0,5
1,16978,2093,3.5,2015-03-31 06:03:17,0,2015-03-31,2015,Return to Oz (1985),Adventure|Children|Fantasy,"[adventure, children, fantasy]",...,0,0,0,1,1,0,0,0,0,1
2,53930,118706,3.5,2015-03-31 06:00:51,0,2015-03-31,2015,Black Sea (2014),Adventure|Thriller,"[adventure, thriller]",...,0,0,0,0,0,0,0,0,0,4
3,16978,106642,3.0,2015-03-31 06:00:28,0,2015-03-31,2015,"Day of the Doctor, The (2013)",Adventure|Drama|Sci-Fi,"[adventure, drama, sci-fi]",...,0,1,0,0,0,0,0,0,0,4
4,70232,58998,2.5,2015-03-31 05:55:28,0,2015-03-31,2015,Forgetting Sarah Marshall (2008),Comedy|Romance,"[comedy, romance]",...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6985821,85252,48,4.0,1996-01-29 00:00:00,1,1996-01-29,1996,Pocahontas (1995),Animation|Children|Drama|Musical|Romance,"[animation, children, drama, musical, romance]",...,1,0,1,0,1,0,0,0,0,1
6985822,85252,50,5.0,1996-01-29 00:00:00,1,1996-01-29,1996,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"[crime, mystery, thriller]",...,0,0,0,0,0,0,0,1,0,7
6985823,85252,60,4.0,1996-01-29 00:00:00,1,1996-01-29,1996,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,"[adventure, children, fantasy]",...,0,0,0,1,1,0,0,0,0,1
6985824,85252,70,4.0,1996-01-29 00:00:00,1,1996-01-29,1996,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,"[action, comedy, horror, thriller]",...,0,0,0,0,0,0,0,0,0,2


Observemos el siguiente ejemplo del usuario 85252 que ha caído en el conjunto de entrenamiento.

In [56]:
ind_userId=85252
data_ind_user_Id = rating_redundant_train_2_with_cluster[rating_redundant_train_2_with_cluster['userId']==ind_userId]
count_cluster_data_ind = len(data_ind_user_Id['cluster'].unique())
count_movies_ind_userId = data_ind_user_Id.shape[0]
display(data_ind_user_Id[['userId', 'movieId', 'high_rating', 'time_day', 'year', 'title', 'genres', 'cluster' ]].head(10))
logging.info(f'EN TOTAL EL USUARIO {ind_userId} TIENE {count_movies_ind_userId} PELÍCULAS. LOS CLUSTERS SON EN TOTAL {count_cluster_data_ind}')

,userId,movieId,high_rating,time_day,year,title,genres,cluster
6443295,85252,1391,0,1996-12-14,1996,Mars Attacks! (1996),Action|Comedy|Sci-Fi,2
6450988,85252,475,0,1996-12-11,1996,In the Name of the Father (1993),Drama,9
6456484,85252,481,0,1996-12-08,1996,Kalifornia (1993),Drama|Thriller,0
6456485,85252,1374,0,1996-12-08,1996,Star Trek II: The Wrath of Khan (1982),Action|Adventure|Sci-Fi|Thriller,2
6463953,85252,1183,1,1996-12-03,1996,"English Patient, The (1996)",Drama|Romance|War,10
6466285,85252,1367,0,1996-11-30,1996,101 Dalmatians (1996),Adventure|Children|Comedy,1
6482524,85252,1356,1,1996-11-23,1996,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,2
6494140,85252,673,0,1996-11-18,1996,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sc...,1
6507330,85252,832,1,1996-11-10,1996,Ransom (1996),Crime|Thriller,7
6512864,85252,1059,1,1996-11-08,1996,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,5


2021-11-28 17:29:36 INFO: EN TOTAL EL USUARIO 85252 TIENE 198 PELÍCULAS. LOS CLUSTERS SON EN TOTAL 14


EL pasado usuario tiene 198 películas reseñadas. Sin embargo, estas películas están contenidas en 14 cluster diferentes. Por lo tanto, se eliminarán todas las filas con repeticiones de un mismo cluster para cada usuario. Esto ayudará a quitar la información redundante de películas y lidiar así con posibles sobreajustes. 

NOTA: Se dará prioridad en traer la observación más reciente sobre ese cluster, es decir, el valor de `high_rating` más reciente. Sin embargo, también se medirá si para ese mismo cluster ha habido un "cambio de opinión" en el pasado, es decir, si en algun momento se pasó de tener un `high_rating` para luego cambiarlo a `low_ratinG` (o viceversa). Por ende, también se trae, para ese mismo usuario, la información más reciente en la haya habido un `rating` diferente para un mismo cluster. Cabe aclarar que este "cambio de opinión" del cluster se está permitiendo que suceda la misma fecha (`time_day`) que el `rating` más reciente; aunque, también se podría imponer la restricción de que no suceda esto el mismo día que la `rating` más reciente, y así, también se tendría que controlar `time_day`, pero este caso no pudo considerarse por falta de tiempo.

In [59]:
rating_train_2 = pd.merge(rating_redundant_train_2,
                          movie_with_cluster,
                          how="left",
                          on=["movieId"])
rating_train_2.reset_index(drop=True,
                           inplace=True)
rating_train_whithout_duplicate = rating_train_2.drop_duplicates(subset=['userId', 'high_rating', 'cluster'])

Para el ejemplo anterior, el usuario se quedaría tan solo con 26 de las 198 películas. Se depliega a continuación.

In [72]:
data_ind = rating_train_whithout_duplicate[rating_train_whithout_duplicate['userId']==ind_userId][['userId', 'movieId', 'high_rating', 'time_day', 'year', 'title', 'genres', 'cluster' ]]
data_ind

,userId,movieId,high_rating,time_day,year,title,genres,cluster
6443295,85252,1391,0,1996-12-14,1996,Mars Attacks! (1996),Action|Comedy|Sci-Fi,2
6450988,85252,475,0,1996-12-11,1996,In the Name of the Father (1993),Drama,9
6456484,85252,481,0,1996-12-08,1996,Kalifornia (1993),Drama|Thriller,0
6463953,85252,1183,1,1996-12-03,1996,"English Patient, The (1996)",Drama|Romance|War,10
6466285,85252,1367,0,1996-11-30,1996,101 Dalmatians (1996),Adventure|Children|Comedy,1
6482524,85252,1356,1,1996-11-23,1996,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,2
6507330,85252,832,1,1996-11-10,1996,Ransom (1996),Crime|Thriller,7
6512864,85252,1059,1,1996-11-08,1996,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,5
6548913,85252,900,0,1996-10-23,1996,"American in Paris, An (1951)",Musical|Romance,14
6548915,85252,928,0,1996-10-23,1996,Rebecca (1940),Drama|Mystery|Romance|Thriller,5


Más aún, para este mismo ejemplo, si se mira cómo se comporta el Cluster 2, se notará lo siquiente.

In [73]:
data_ind[data_ind.cluster==2]

,userId,movieId,high_rating,time_day,year,title,genres,cluster
6443295,85252,1391,0,1996-12-14,1996,Mars Attacks! (1996),Action|Comedy|Sci-Fi,2
6482524,85252,1356,1,1996-11-23,1996,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,2


En distintos momentos, este usuario cambio de `rating` 1 a 0 para las películas del tipo Cluster 2.

In [67]:
logging.info('SE ELIMINA REDUNDANCIA EN LA INFORMACIÓN DE PELÍCULAS DEL ENTRENAMIENTO.')
rating_train_whithout_duplicate

2021-11-28 17:40:13 INFO: SE ELIMINA REDUNDANCIA EN LA INFORMACIÓN DE PELÍCULAS DEL ENTRENAMIENTO.


,userId,movieId,rating,timestamp,high_rating,time_day,year,title,genres,genres_list,...,genre_musical,genre_sci-fi,genre_animation,genre_fantasy,genre_children,genre_western,genre_war,genre_mystery,genre_film-noir,cluster
0,87586,7151,3.5,2015-03-31 06:40:02,0,2015-03-31,2015,Girl with a Pearl Earring (2003),Drama|Romance,"[drama, romance]",...,0,0,0,0,0,0,0,0,0,5
1,16978,2093,3.5,2015-03-31 06:03:17,0,2015-03-31,2015,Return to Oz (1985),Adventure|Children|Fantasy,"[adventure, children, fantasy]",...,0,0,0,1,1,0,0,0,0,1
2,53930,118706,3.5,2015-03-31 06:00:51,0,2015-03-31,2015,Black Sea (2014),Adventure|Thriller,"[adventure, thriller]",...,0,0,0,0,0,0,0,0,0,4
3,16978,106642,3.0,2015-03-31 06:00:28,0,2015-03-31,2015,"Day of the Doctor, The (2013)",Adventure|Drama|Sci-Fi,"[adventure, drama, sci-fi]",...,0,1,0,0,0,0,0,0,0,4
4,70232,58998,2.5,2015-03-31 05:55:28,0,2015-03-31,2015,Forgetting Sarah Marshall (2008),Comedy|Romance,"[comedy, romance]",...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6985753,124035,73,4.0,1996-02-01 14:34:07,1,1996-02-01,1996,"Misérables, Les (1995)",Drama|War,"[drama, war]",...,0,0,0,0,0,0,1,0,0,10
6985763,124035,24,3.0,1996-02-01 14:33:54,0,1996-02-01,1996,Powder (1995),Drama|Sci-Fi,"[drama, sci-fi]",...,0,1,0,0,0,0,0,0,0,4
6985774,124035,22,4.0,1996-02-01 14:33:44,1,1996-02-01,1996,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,"[crime, drama, horror, mystery, thriller]",...,0,0,0,0,0,0,0,1,0,7
6985784,124035,61,4.0,1996-02-01 14:33:34,1,1996-02-01,1996,Eye for an Eye (1996),Drama|Thriller,"[drama, thriller]",...,0,0,0,0,0,0,0,0,0,0


Aplicando lo anterior a todos los usuarios de entrenamiento, para el conjunto `rating_redundant_train_2` de 6,985,826 observaciones se pasa a tener un conjunto de 1,002,389, llamado `rating_train_whithout_duplicate`. Con este último conjunto, el reducido, se modelará más adelante. A continuación se depliegan sus insights.

In [75]:
users_train_2 = rating_train_whithout_duplicate['userId'].unique()
count_users_train_2 = len(users_train_2 )
movies_train_2 = rating_train_whithout_duplicate['movieId'].unique()
count_movies_train_2 = len(movies_train_2 )
logging.info(f'TOTAL DE DATA EN ENTRENAMIENTO SIN REDUNDANCIAS: {rating_train_whithout_duplicate.shape[0]}')
logging.info(f'TOTAL DE PELÍCULAS EN ENTRENAMIENTO SIN REDUNDANCIAS: {count_movies_train_2}')
logging.info(f'TOTAL DE USUARIOS EN ENTRENAMIENTO SIN REDUNDANCIAS: {count_users_train_2}')

2021-11-28 18:14:01 INFO: TOTAL DE DATA EN ENTRENAMIENTO SIN REDUNDANCIAS: 1002389
2021-11-28 18:14:01 INFO: TOTAL DE PELÍCULAS EN ENTRENAMIENTO SIN REDUNDANCIAS: 13901
2021-11-28 18:14:01 INFO: TOTAL DE USUARIOS EN ENTRENAMIENTO SIN REDUNDANCIAS: 48473


In [76]:
logging.info('SE GUARDA LA NUEVA DATA DE ENTRENAMIENTO SIN REDUNDANCIAS.')
# data_path='rating_train_whithout_duplicate.sav'
# pickle.dump(rating_train_whithout_duplicate, open(data_path, 'wb'))

2021-11-28 18:14:53 INFO: SE GUARDA LA NUEVA DATA DE ENTRENAMIENTO SIN REDUNDANCIAS.


# Features para codificación de los Usuarios.

Para los usuarios, se ha aprovechado la historia indexada en el tiempo de cada uno de ellos. Se ha utilizado las fechas en la que se otorgaron los distintos `rating`'s, es decir, los `time_day`'s del usuario. Como puede haber varias películas revisadas por el usuario en un mismo día, lo que se hace es una agrupación por fechas. Esto último es la base para realizar conteos y acumulados. Solo ha dado tiempo de calcular los más intuitivos.

La construcción de estas features se hace através de una script de python llamado `feature_enginering_user.py` el cual contiene la función `total_past_extractor`, la cual realiza la tarea globlamente para un *conjunto de datos redundante*. Se depende a su vez del script `past_information_user.py`, la cual contiene una función homónima que realiza las construcciones de manera individual, es decir, por usuario.

La construcción se hace de la siguiente manera. Dado un `time_day` en la que un usuario revisó una o varias películas, se obtuvo:
* `frecc_movies_per_day`: la frecuencia de películas en ese `time_day`.
* `frecc_genre_crime_per_day`: la frecuencia de películas del género *crime* en ese `time_day`.
* `frecc_genre_drama_per_day`: la frecuencia de películas del género *drama* en ese `time_day`.
* y así sucesivamente hasta abarcar todos los géneros.

Esto se hace para todos los `time_day` en el historial del usuario. Lo anterior intenta codificar la información del usuario que se tiene "en ese instante", mas bien, la que se tiene en el día que va a otorgar el `rating`.

Una vez hecho esto, y usando los `time_day` anteriores al actual, se puede dar pie a los acumulados que se toman desde fechas pasadas. Con esto, se tiene lo siguiente:

* `cumulativefrecc_movies_per_day`: la frecuencia acumulada de películas hasta el `time_day` actual.
* `cumulativefrecc_genre_crime_per_day`: la frecuencia acumulada de películas del género *crime* hasta el `time_day` actual.
* `cumulativefrecc_genre_drama_per_day`: la frecuencia acumulada de películas del género *drama* hasta el `time_day` actual.
* y así sucesivamente hasta abarcar todos los géneros.

Esto itenta codificar la historia que se tiene del usuario hasta el dia en que va a otorga el `rating`.

Una vez calculados los acumulados que se tienen en los géneros, uno puede dividirlos entre el respectivo `cumulativefrecc_movies_per_day`, para obtener la tendencia de revisión del usuario a través de los disntintos géneros hasta el actual `time_day`. Así se tienen:
* `percencumulativefrecc_genre_crime_per_day`,
* `percencumulativefrecc_genre_drama_per_day`,
* `percencumulativefrecc_genre_adventure_per_day`,
* y así sucesivamente hasta abarcar todos los géneros.

Esto predende medir el cambio de preferencias de géneros revisados del usuario a lo largo de su historia hasta el `time_day` actual.

NOTA: Esta manera de construir estas features está incluyendo a la película actual, es decir, a la película con ese `time_day`, la cual, es muy probable que pueda estarla compartiendo con más películas. Para que esto tenga sentido, se necesita hacer un __supuesto__ más, el cual consiste en que se está considerando que el usuario *YA VIÓ* la película y se está en espera de que le otorque el `rating`. Así, tiene sentido incluir la información de la propia película en estas construcciones.

# Obtención de la data de prueba peculiar.

Se guarda la data con características peculiares, es decir, aquella con usuarios en el conjunto de prueba con películas fuera del entrenamiento.

In [83]:
# peculiar_data_test = rating_redundant_test_1[rating_redundant_test_1['movieId'].isin(movies_not_in_train)]
# data_path = 'peculiar_data_test.sav'
# pickle.dump(peculiar_data_test,
#             open(data_path, 'wb'))